## FPGA #1

In [1]:
import pynq
import numpy as np
from _thread import *
import threading 
import socket

In [2]:
import pynq, os, sys
sys.path.append('../')
import vnx_utils

In [3]:
for i in range(len(pynq.Device.devices)):
    print("{}) {}".format(i, pynq.Device.devices[i].name))

0) xilinx_u200_gen3x16_xdma_base_1


In [4]:
currentDevice = pynq.Device.devices[0]
xclbin = 'gamma_filter/build_dir.hw.xilinx_u200_gen3x16_xdma_1_202110_1/gamma_filter.xclbin'
ol = pynq.Overlay(xclbin,device=currentDevice)

In [5]:
ol.ip_dict

{'gamma_filter_1': {'phys_addr': 16842752,
  'addr_range': 4096,
  'type': 'xilinx.com:hls:gamma_filter:1.0',
  'hw_control_protocol': 'ap_ctrl_chain',
  'fullpath': 'gamma_filter_1',
  'registers': {'CTRL': {'address_offset': 0,
    'access': 'read-write',
    'size': 4,
    'description': 'OpenCL Control Register',
    'type': 'unsigned int',
    'id': None,
    'fields': {'AP_START': {'access': 'read-write',
      'bit_offset': 0,
      'bit_width': 1,
      'description': 'Start the accelerator'},
     'AP_DONE': {'access': 'read-only',
      'bit_offset': 1,
      'bit_width': 1,
      'description': 'Accelerator has finished - cleared on read'},
     'AP_IDLE': {'access': 'read-only',
      'bit_offset': 2,
      'bit_width': 1,
      'description': 'Accelerator is idle'},
     'AP_READY': {'access': 'read-only',
      'bit_offset': 3,
      'bit_width': 1,
      'description': 'Accelerator is ready to start next computation'},
     'AUTO_RESTART': {'access': 'read-write',
      

### Receive data from FPGA manager

In [6]:
import socket
from struct import unpack

def start_server(host, port):
    # host = '10.1.212.147'
    # port = 64505  # Make sure this port is not in use

    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(1)
    print('Server started. Waiting for connections...')
    server_running = True
    
    while server_running:
        client_socket, addr = server_socket.accept()
        print(f'Connection from: {str(addr)}')
        while True:
            bs = client_socket.recv(8)
            (length,) = unpack('>Q', bs)
            print(length)
            recv_data = b''
            while len(recv_data) < length:
                to_read = length - len(recv_data)
                recv_data += client_socket.recv(
                    4096 if to_read > 4096 else to_read)
            # recv_data = receive_all(client_socket, image_data_size)
            if not recv_data:
                break
            print(f'From connected user: {recv_data[:1024]} ...')
            data = '202'
            client_socket.send(data.encode())
            
            server_running = False
            break

        client_socket.close()
    server_socket.close()
    print("close connection")
    return recv_data

In [ ]:
gamma = start_server('10.1.212.148', 64502)

Server started. Waiting for connections...


In [ ]:
import re
s = re.sub(r'\[|\]', '', gamma.decode())
data = list(map(int, s.split()))
print(data)

In [ ]:
krnl_gamma_filter = ol.gamma_filter_1

In [ ]:
in_size = 2
out_size = 256
in_shape = (in_size, 1)
out_shape = (out_size, 1)
if hasattr(ol, 'HBM0'):
    rt_buf_in = pynq.allocate(in_shape, dtype=np.uint8, target=ol.HBM0)
    rt_buf_out = pynq.allocate(out_shape, dtype=np.uint8, target=ol.HBM0)
else:
    rt_buf_in = pynq.allocate(in_shape, dtype=np.uint8, target=ol.bank1)
    rt_buf_out = pynq.allocate(out_shape, dtype=np.uint8, target=ol.bank1)

In [ ]:
data=[1, 80]
rt_buf_in[:] = np.reshape(np.array(data), in_shape)
rt_buf_in.sync_to_device()

In [ ]:
rt_hw = krnl_gamma_filter.start(rt_buf_in, rt_buf_out)

In [ ]:
rt_hw.wait()
rt_buf_out.sync_from_device()

In [ ]:
# print(rt_buf_out)
output_shape = (1, out_size)
filter = np.reshape(np.array(rt_buf_out), output_shape)
print(filter)

In [ ]:
%xdel rt_buf_in
%xdel rt_buf_out
ol.free()

### sending the data to the next FPGA

In [ ]:
from struct import pack

def start_client(host, port, message):
    # host = '10.1.212.148'
    # port = 64501  # Must be the same as the server port

    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((host, port))
    message = message.encode()
    length = pack('>Q', len(message)) 
    print(length)
    client_socket.sendall(length) #send length
    client_socket.sendall(message) #send data
    #receive resp from server
    data = client_socket.recv(3)
    print(f'Received from server: {data}')
    
    client_socket.close()
    return data

In [ ]:
import time 
time.sleep(2)

In [ ]:
data = np.array2string(filter)
recv = start_client('10.1.212.147', 64508, data)

In [ ]:
print(len(data))